In [1]:
from datetime import datetime
import sys
import time
# to import local fuctions
sys.path.insert(0, '../tools')

from multiprocessing import Pool, cpu_count

from extract.price_extract import *
from extract.screener import finviz_pull
from indicators.indicators import *
from wrapper import main

from database.postgres import PostgresTable, create_new_table

import sqlalchemy as db
import pandas as pd
from yahooquery import Ticker

In [2]:
stocks_table = PostgresTable('postgresql://localhost/swing_trading', 'raw_finviz_pull')
prices_table = PostgresTable('postgresql://localhost/swing_trading', 'raw_prices')
indicators_table = PostgresTable('postgresql://localhost/swing_trading', 'indicators')

In [4]:
pool = Pool(3)

In [7]:
pool.terminate()

In [3]:
main(stocks_table, prices_table, indicators_table, skip_price=True)

ValueError: No objects to concatenate

In [29]:
main(stocks_table, prices_table, indicators_table)

2097 stocks will be analyzed

Start Update Prices Process ...
raw_prices is updated!
It took 737 seconds to extract prices.



ValueError: No objects to concatenate

In [ ]:
ind_df = indicators_parallel(stocks_list, price_df)

In [7]:
# Extract prices and symbols from tables
price_df = prices_table.read_table_to_pandas(drop_upload_date=True)
stocks_list = stocks_table.extract_column_to_list('symbol')[:100]

KeyboardInterrupt: 

In [8]:
# Run main indicators without any filters
indicators_df = indicators_parallel(stocks_list, price_df)

In [3]:
main_indicators(prices_table, stocks_table, indicators_table)

In [ ]:
ret = indicators_table.remove_duplicates(id_column='id', 
                                         partition_by_columns='symbol, date', 
                                         order_by_column='upload_datetime')

In [17]:
x = run_indicators(price_df, 'XXXXX')

In [21]:
lst

[None, None]

In [22]:
x == None

True

In [20]:
#lst = [] 
lst.append(x)

In [ ]:
len(indicators_table.read_table_to_pandas())

In [ ]:
len(indicators_table.read_table_to_pandas())

In [ ]:
indicators_df = indicators_table.read_table_to_pandas()

In [ ]:
indicators_df[indicators_df['symbol'] == 'SINA'].sort_values('date')

In [ ]:
coke_df = pd.read_csv('/Users/reno/Downloads/indicators_202103311131.csv')
coke_df = coke_df.astype(str)
coke_df['unique_id'] = coke_df[coke_df.columns].agg('-'.join, axis=1)
id_count = coke_df.groupby('id').count()['symbol'].reset_index()
id_count.columns = ['id', 'id_count']
unique_id_count = coke_df.groupby('unique_id').count()['symbol'].reset_index()
unique_id_count.columns = ['unique_id', 'unique_id_count']
coke_df = pd.merge(coke_df, id_count, on='id')
coke_df = pd.merge(coke_df, unique_id_count, on='unique_id')
coke_df[(coke_df['id_count'] == 2)].to_csv('test.csv')

In [ ]:
lst = []
run_indicators(price_df, lst, 'AMD', stocks_table.connection)

In [ ]:
start_time = time.time()
main_prices(stocks_table, prices_table)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
indicators_df.to_sql(name='indicators', con=prices_table.connection, if_exists= 'replace', index=False)

In [ ]:
prices_table.connection

In [ ]:
prices_df.head()

In [ ]:
results = []
run_indicators(prices_df, results, 'AAPL')

In [ ]:
def get_count(values):
    return len(values)

grouped_count = df.groupby("my_label").agg(get_count)

In [ ]:
results[0]

In [ ]:
dicttt = stock_price_min_max_dates(prices_table)

In [ ]:
lst = []
for stock, values in dicttt.items():
    lst.append(values['min_date'] + values['min_date'])
    

In [ ]:
dict_df = pd.DataFrame(dicttt).transpose().reset_index()
dict_df.sort_values('max_date')
#dict_df.groupby(['min_date', 'max_date']).count()['index']

In [ ]:
main_prices(stocks_table, prices_table)

In [ ]:
finviz_pull(stocks_table)

In [ ]:
#stocks_table.read_table_to_pandas()
df = prices_table.read_table_to_pandas()
df[df['symbol'] == 'EBR-B']

In [ ]:
df[df['symbol'] == 'EBR-B'].sort_values('date')

In [ ]:
#stocks_table.load_data(df, id_columns=['symbol'], is_replace=True)
stocks_table.read_table_to_pandas()

### Create Engine

In [ ]:
def postgre_engine(postgresql_url):
    """
    Function to create PostgreSQL Database Engine, Metadata and Connection.
    
    Arguments:
    - postgresql_url(str): PostgreSQL URL to access the database.
    
    Return:
    - engine
    - connection
    - metadata
    """
    # Create Engine Connecting to DataBase
    engine = db.create_engine(postgresql_url)
    connection = engine.connect()
    metadata = db.MetaData()
    
    return engine, connection, metadata

### Load Data Function

In [ ]:
def load_data(postgresql_url, table_name, data_to_load, is_replace):
    """
    Function to load data into a PostgreSQL table if columns and table exist.
    
    Arguments:
    - postgresql_url(str): PostgreSQL URL to access the database.
    - table_name(str): Table Name in the database.
    - data_to_load(list): List of dictionaries with values to upload, one key per column in table.
    - is_replace(boolean): Indicates whether it should completely replace the table and erase all data.
    
    Return:
    - result(sqlalchemy.engine.result): Result after running the query.
    """
    # Define upload time
    upload_time = datetime.now().strftime('%d/%m/%y %H:%M:%S')
    
    # Add key with upload time
    for data_row in data_to_load:
        data_row['upload_datetime'] = upload_time
    
    # Create Engine Connecting to DataBase
    engine, connection, metadata = postgre_engine(postgresql_url)
 
    # Tables in databaset
    tables_in_db = engine.table_names()
    
    # Check if table is in the database
    if table_name in tables_in_db:

        # Create instance of Table
        table = db.Table(table_name, metadata, autoload=True, autoload_with=engine)

        # Check if columns match
        # Pull keys from database
        query = 'select * from ' + table_name + ' limit 1'
        table_cols = connection.execute(query).keys()

        # Get list of keys from first item in dictionary
        to_load_cols = list(data_to_load[0].keys())
        
        # Get differences between columns
        table_cols_diff = set(table_cols) - set(to_load_cols)
        to_load_cols_diff = set(to_load_cols) - set(table_cols)

        # If columns match, then we are good
        if table_cols == to_load_cols:

            # If the method is 'replace', then first it will drop all data in table
            if is_replace == True:
                # Drop all data from table
                query_delete = db.delete(table)
                connection.execute(query_delete)

            # Insert data to table
            query_result = connection.execute(db.insert(table), data_to_load)
            
            return query_result

        else:
            print('Columns do not match!')
            print('In Table but not in data: {}'.format(table_cols_diff))
            print('In Data but not in Table: {}'.format(to_load_cols_diff))
    
    else:
        print('{} does not exist in the database!'.format(table_name))

In [ ]:
load_data('postgresql://localhost/swing_trading', 'raw_finviz_pull', stocks_upload, is_replace=True)

In [ ]:
load_data('postgresql://localhost/swing_trading', 'raw_prices', prices_upload, is_replace=True)

### Read All data from Table

In [ ]:
def read_table_to_pandas(postgresql_url, table_name):
    # Create Engine Connecting to DataBase
    engine, connection, metadata = postgre_engine(postgresql_url)

    # Create instance of Table
    table = db.Table(table_name, metadata, autoload=True, autoload_with=engine)
    
    # Pull all data from table and return it as a Pandas DataFrame
    query = 'select * from {}'.format(table_name)
    df_query = pd.read_sql_query(sql, engine)
    
    return df_query

### Create New Table

In [ ]:
# Create Engine Connecting to Swing Trading DB
engine, connection, metadata = postgre_engine('postgresql://localhost/swing_trading')
new_table = db.Table('raw_finviz_pull', metadata,
                db.Column('symbol', db.String(), nullable=False),
                db.Column('description', db.String(), nullable=False),
                db.Column('industry', db.String(), nullable=False),
                db.Column('sector', db.String(), nullable=False),
                db.Column('market_cap', db.String(), nullable=False),
                db.Column('upload_datetime', db.String(), nullable=False),
                )

metadata.create_all(engine)

### Add Single Record

In [ ]:
# stmt = db.insert(emp).values(ticker='test2', name='test', industry='test', sector='test',
#                              market_cap='test')
# result_proxy = connection.execute(stmt)

### Delete all rows

In [ ]:
# stmt = db.delete(emp)
# result_proxy = connection.execute(stmt)

### Drop Table

In [ ]:
# Create instance with raw_finviz_pull
engine = db.create_engine('postgresql://localhost/swing_trading')
connection = engine.connect()
metadata = db.MetaData()
#raw_finviz = db.Table('raw_prices', metadata, autoload=True, autoload_with=engine)
#raw_finviz.drop(engine)

### Table Names

In [ ]:
engine.table_names()

### Query Data

In [ ]:
sql = 'select * from raw_prices'
pd.read_sql_query(sql, engine)

### Recreate Tables using csv

In [ ]:
table_reference = pd.read_csv('../docs/tables.csv')
raw_prices_table_ref = table_reference[table_reference['table_name'] == 'raw_finviz_pull']
raw_prices_table_ref

In [ ]:
create_new_table('postgresql://localhost/swing_trading', 'raw_prices', drop_current_table=True)

In [ ]:
min_max_dates = stock_price_min_max_dates(prices_table)

In [ ]:
results = []
run_prices_list(['IBM', 'AAPL', 'CGC'], results, min_max_dates)

In [ ]:
pd.concat(results)